In [1]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/108.6 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 3.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.9 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.9 MB/s eta 0:00:00


...

 \

 |

 done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114330 sha256=6aa2d2c82e98428ffe15114823e5c8770b7ed124edc534aed24523f67ce97554
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
from deepface import DeepFace
from sklearn.metrics import accuracy_score, precision_score, recall_score
import tensorflow as tf
import cv2

E0000 00:00:1751441001.586058      74 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


25-07-02 07:23:37 - Directory /root/.deepface has been created


25-07-02 07:23:37 - Directory /root/.deepface/weights has been created


In [3]:
# Define paths (adjust according to your dataset and model location)
VAL_DATASET_PATH = "/kaggle/input/validation/val"  # Adjusted for nesting
MODEL_PATH = "/kaggle/input/facerecognizer/keras/default/1/siamese_face_matching.h5"  # Default path; update if using a dataset
if not os.path.exists(MODEL_PATH):
    print(f"Model not found at {MODEL_PATH}. Check path or regenerate model.")
    MODEL_PATH = "../input/your-output-dataset/siamese_face_matching.h5"  # Adjust this path if using a dataset
IDENTITY_FOLDERS_VAL = [f for f in os.listdir(VAL_DATASET_PATH) if os.path.isdir(os.path.join(VAL_DATASET_PATH, f))]

In [4]:
# Function to load clear and distorted images and extract embeddings
def load_and_embed_images(folder_path):
    clear_embeddings = []
    clear_labels = []
    distorted_embeddings = []
    distorted_names = []
    distorted_paths = []
    
    for identity in IDENTITY_FOLDERS_VAL:
        if os.path.isdir(os.path.join(folder_path, identity)):
            identity_path = os.path.join(folder_path, identity)
            # Load clear reference image
            clear_images = [f for f in os.listdir(identity_path) if f.endswith(('.jpg', '.jpeg', '.png')) and not os.path.isdir(os.path.join(identity_path, f))]
            if clear_images:
                clear_img_path = os.path.join(identity_path, clear_images[0])
                try:
                    clear_embedding_dicts = DeepFace.represent(clear_img_path, model_name="VGG-Face", enforce_detection=False)
                    if clear_embedding_dicts:  # Check if list is not empty
                        clear_embedding = clear_embedding_dicts[0]['embedding']  # Take first embedding
                        clear_embeddings.append(clear_embedding)  # Append as 1D array
                        clear_labels.append(identity)
                    else:
                        print(f"No embeddings found for clear image {clear_img_path}")
                except Exception as e:
                    print(f"Error processing clear image {clear_img_path}: {e}")
            
            # Load distorted images from the nested distorted folder
            distorted_folder = os.path.join(identity_path, "distortion")
            if os.path.isdir(distorted_folder):
                for dist_img_name in os.listdir(distorted_folder):
                    dist_img_path = os.path.join(distorted_folder, dist_img_name)
                    try:
                        dist_embedding_dicts = DeepFace.represent(dist_img_path, model_name="VGG-Face", enforce_detection=False)
                        if dist_embedding_dicts:  # Check if list is not empty
                            dist_embedding = dist_embedding_dicts[0]['embedding']  # Take first embedding
                            distorted_embeddings.append(dist_embedding)  # Append as 1D array
                            distorted_names.append(dist_img_name)
                            distorted_paths.append(dist_img_path)
                        else:
                            print(f"No embeddings found for distorted image {dist_img_path}")
                    except Exception as e:
                        print(f"Error processing distorted image {dist_img_path}: {e}")
    
    # Convert to 2D arrays, ensuring consistent shape
    clear_embeddings = np.array(clear_embeddings)
    if clear_embeddings.size == 0:
        clear_embeddings = np.array([]).reshape(0, 0)  # Handle empty array
    elif clear_embeddings.ndim == 1:
        clear_embeddings = clear_embeddings.reshape(-1, clear_embeddings.shape[0])
    
    distorted_embeddings = np.array(distorted_embeddings)
    if distorted_embeddings.size == 0:
        distorted_embeddings = np.array([]).reshape(0, 0)  # Handle empty array
    elif distorted_embeddings.ndim == 1:
        distorted_embeddings = distorted_embeddings.reshape(-1, distorted_embeddings.shape[0])
    
    return clear_embeddings, clear_labels, distorted_embeddings, distorted_names, distorted_paths

In [5]:
# Function to compute distance using the Siamese model
def compute_distance(model, emb1, emb2):
    return model.predict([np.array([emb1]), np.array([emb2])], verbose=0)[0][0]

In [6]:
# Main test function
def main():
    # Load the trained Siamese model
    try:
        siamese_model = tf.keras.models.load_model(MODEL_PATH, custom_objects={"contrastive_loss": lambda y_true, y_pred: y_pred})
        print(f"Model loaded successfully from {MODEL_PATH}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Load validation data
    clear_embeddings, clear_labels, distorted_embeddings, distorted_names, distorted_paths = load_and_embed_images(VAL_DATASET_PATH)

    # Perform matching
    predictions = []
    true_labels = []

    for dist_emb, dist_name, dist_path in zip(distorted_embeddings, distorted_names, distorted_paths):
        min_distance = float("inf")
        predicted_identity = None

        for ref_emb, ref_label in zip(clear_embeddings, clear_labels):
            distance = compute_distance(siamese_model, dist_emb, ref_emb)
            if distance < min_distance:
                min_distance = distance
                predicted_identity = ref_label

        # Determine true label from the parent folder name
        true_label = os.path.basename(os.path.dirname(os.path.dirname(dist_path)))
        match_label = 1 if predicted_identity == true_label else 0
        predictions.append(match_label)
        true_labels.append(1)  # Ground truth is 1 for correct match in verification

        print(f"Image: {dist_name}, Predicted Identity: {predicted_identity}, Distance: {min_distance:.4f}, Match: {match_label}")

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average="weighted", zero_division=0)
    recall = recall_score(true_labels, predictions, average="weighted", zero_division=0)

    print("\nTest Results on Validation Set:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

In [7]:
if __name__ == "__main__":
    main()

I0000 00:00:1751441023.415391      74 service.cc:148] XLA service 0x570ff1c206f0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751441023.415436      74 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1751441023.415440      74 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1751441023.415442      74 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1751441023.415445      74 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1751441023.415448      74 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1751441023.415450      74 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1751441023.415453      74 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1751441023.415455      74 service.cc:156]   StreamExecutor de

vice (7): TPU, 2a886c8


Error loading model: bad marshal data (unknown type code)
